In [1]:
import cv2
import dlib
import numpy as np

# Inicializar el detector de rostros de dlib y el predictor de puntos faciales
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("C:/Users/noelp/anaconda3/envs/vc_env/shape_predictor_68_face_landmarks.dat")

# Iniciar la captura de video desde la webcam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

# Configurar VideoWriter para guardar el video de salida en formato MP4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('resultado_video.mp4', fourcc, 20.0, (640, 480))

# Lista para almacenar las bolas de energía generadas
bolas_energia = []

# Función para calcular la distancia entre dos puntos (usada para medir la apertura de la boca)
def calcular_distancia(punto1, punto2):
    return np.linalg.norm(np.array(punto2) - np.array(punto1))

# Función para aplicar un efecto de distorsión en la región de la cara si la boca está abierta
def aplicar_distorsion(frame, landmarks, aumento_tamano=1.2):
    puntos_face = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)])
    
    # Obtener las coordenadas extremas de la cara
    x_min, y_min = np.min(puntos_face, axis=0)
    x_max, y_max = np.max(puntos_face, axis=0)

    # Asegurarse de que las coordenadas estén dentro de los límites de la imagen
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(frame.shape[1] - 1, x_max)
    y_max = min(frame.shape[0] - 1, y_max)

    # Extraer la región de la cara y redimensionarla para crear el efecto de distorsión
    face_region = frame[y_min:y_max, x_min:x_max]
    face_width = int((x_max - x_min) * aumento_tamano)
    face_height = int((y_max - y_min) * aumento_tamano)
    face_region = cv2.resize(face_region, (face_width, face_height))

    # Reposicionar la región distorsionada en el marco original, asegurando que no se salga de los límites
    new_x = max(x_min - (face_width - (x_max - x_min)) // 2, 0)
    new_y = max(y_min - (face_height - (y_max - y_min)) // 2, 0)
    
    # Ajustar el tamaño si se sale de los límites del marco
    new_x_end = min(new_x + face_width, frame.shape[1])
    new_y_end = min(new_y + face_height, frame.shape[0])
    face_region = face_region[:new_y_end-new_y, :new_x_end-new_x]

    # Insertar la región distorsionada de nuevo en el marco original
    frame[new_y:new_y + face_region.shape[0], new_x:new_x + face_region.shape[1]] = face_region


# Función para generar y animar bolas de energía cuando se detecta que la boca está abierta
def generar_bolas_energia(frame, boca_center, distancia_boca):
    global bolas_energia

    # Definir umbrales para diferentes colores de bolas
    umbral_amarillo = 10
    umbral_naranja = 20
    umbral_rojo = 30
    umbral_verde = 40

    # Si la boca está lo suficientemente abierta, crear una nueva bola de energía
    if distancia_boca > umbral_amarillo:
        # Determinar el color de la bola según el tamaño de la apertura de la boca
        if distancia_boca > umbral_verde:
            color = (0, 255, 0)  # Verde si la boca está muy abierta (nivel más alto)
        elif distancia_boca > umbral_rojo:
            color = (0, 0, 255)  # Rojo si la boca está considerablemente abierta
        elif distancia_boca > umbral_naranja:
            color = (0, 165, 255)  # Naranja si la boca está medianamente abierta
        else:
            color = (0, 255, 255)  # Amarillo si la boca está ligeramente abierta
        
        # Crear la bola de energía con características basadas en la apertura
        bola = {
            'center': boca_center,
            'radius': int(10 + distancia_boca // 3),  # Tamaño de la bola
            'direction': np.random.uniform(0, 2 * np.pi),
            'speed': int(3 + distancia_boca // 7),  # Velocidad de la bola
            'color': color
        }
        bolas_energia.append(bola)

    # Mover y dibujar las bolas de energía
    for bola in bolas_energia[:]:
        dx = int(np.cos(bola['direction']) * bola['speed'])
        dy = int(np.sin(bola['direction']) * bola['speed'])
        bola['center'] = (bola['center'][0] + dx, bola['center'][1] + dy)

        # Dibujar la bola de energía en el marco actual con su color correspondiente
        cv2.circle(frame, bola['center'], bola['radius'], bola['color'], -1)

        # Eliminar la bola si sale del área visible
        if not (0 <= bola['center'][0] < frame.shape[1] and 0 <= bola['center'][1] < frame.shape[0]):
            bolas_energia.remove(bola)



# Bucle principal para procesar cada fotograma del video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convertir el fotograma a escala de grises para la detección de rostros
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # Procesar cada rostro detectado
    for face in faces:
        landmarks = predictor(gray, face)
        boca_superior = (landmarks.part(62).x, landmarks.part(62).y)
        boca_inferior = (landmarks.part(66).x, landmarks.part(66).y)
        distancia_boca = calcular_distancia(boca_superior, boca_inferior)
        boca_center = (landmarks.part(62).x, landmarks.part(62).y)

        # Aplicar distorsión si la boca está abierta
        if distancia_boca > 20:
            aplicar_distorsion(frame, landmarks)

        # Generar bolas de energía al abrir la boca
        generar_bolas_energia(frame, boca_center, distancia_boca)

    # Mostrar el fotograma con los efectos aplicados
    cv2.imshow('Filtro Animado', frame)

    # Guardar el fotograma en el archivo de video de salida
    out.write(frame)

    # Salir si se presiona la tecla 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos al finalizar
cap.release()
out.release()
cv2.destroyAllWindows()